In [1]:
import sqlite3

%load_ext sql
%sql sqlite:///library2.db

Connecting to 'sqlite:///library2.db'

In [2]:
%%sql

PRAGMA foreign_keys = ON;

-- LibraryItem Table
CREATE TABLE LibraryItem (
    itemID INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    type TEXT NOT NULL,
    publicationDate DATE,
    authorFirstName TEXT,
    authorLastName TEXT,
    isFutureAcq BOOLEAN DEFAULT 0, -- Boolean for future acquisition
    expectedAcquisitionDate DATE,
    FOREIGN KEY (authorFirstName, authorLastName) REFERENCES Author(firstName, lastName)
);

-- Author Table
CREATE TABLE Author (
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    PRIMARY KEY (firstName, lastName)
);

-- Loan Table
CREATE TABLE Loan (
    loanCard INTEGER,
    itemID INTEGER,
    loanDateTime DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    dueDate DATE NOT NULL,
    returnDate DATE,
    renewalCount INTEGER DEFAULT 0,
    PRIMARY KEY (loanCard, itemID, loanDateTime),
    FOREIGN KEY (loanCard) REFERENCES User(libraryCardNumber) ON DELETE CASCADE,
    FOREIGN KEY (itemID) REFERENCES LibraryItem(itemID) ON DELETE CASCADE
);

-- User Table
CREATE TABLE User (
    libraryCardNumber INTEGER PRIMARY KEY,
    address TEXT NOT NULL,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phoneNumber TEXT UNIQUE NOT NULL
);

-- LibraryStaff Table
CREATE TABLE LibraryStaff (
    staffID INTEGER PRIMARY KEY,
    staffCardNumber INTEGER UNIQUE NOT NULL,
    hireDate DATE NOT NULL,
    salary DECIMAL(10,2) CHECK (salary >= 0),
    FOREIGN KEY (staffCardNumber) REFERENCES User(libraryCardNumber) ON DELETE CASCADE
);

-- Fine Table
CREATE TABLE Fine (
    fineLibraryCard INTEGER,
    dateTimeIssued DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    amount DECIMAL(10,2) CHECK (amount > 0),
    paid BOOLEAN DEFAULT 0,
    PRIMARY KEY (fineLibraryCard, dateTimeIssued),
    FOREIGN KEY (fineLibraryCard) REFERENCES User(libraryCardNumber) ON DELETE CASCADE
);

-- Event Table
CREATE TABLE Event (
    title TEXT NOT NULL,
    dateTime DATETIME NOT NULL,
    roomNum INTEGER,
    numAttendees INTEGER DEFAULT 0,
    maxAttendees INTEGER CHECK (maxAttendees >= 0),
    duration INTEGER CHECK (duration > 0),
    registrationEnd DATETIME,
    description TEXT,
    PRIMARY KEY (title, dateTime),
    FOREIGN KEY (roomNum) REFERENCES LibraryRoom(RoomNumber) ON DELETE SET NULL
);

-- LibraryRoom Table
CREATE TABLE LibraryRoom (
    RoomNumber INTEGER PRIMARY KEY,
    capacity INTEGER CHECK (capacity > 0)
);

-- EventAttendees Table
CREATE TABLE EventAttendees (
    libraryCardNumber INTEGER NOT NULL,
    eventTitle TEXT NOT NULL,
    eventDateTime DATETIME NOT NULL,
    PRIMARY KEY (libraryCardNumber, eventTitle, eventDateTime),
    FOREIGN KEY (libraryCardNumber) REFERENCES User(libraryCardNumber) ON DELETE CASCADE,
    FOREIGN KEY (eventTitle, eventDateTime) REFERENCES Event(title, dateTime) ON DELETE CASCADE
);



Running query in 'sqlite:///library2.db'

++
||
++
++

In [3]:
%%sql
INSERT INTO Author (firstName, lastName) VALUES
('John', 'Smith'),
('Mary', 'Johnson'),
('James', 'Williams'),
('Patricia', 'Brown'),
('Michael', 'Jones'),
('Linda', 'Garcia'),
('David', 'Martinez'),
('Susan', 'Hernandez'),
('Robert', 'Lopez'),
('Elizabeth', 'Gonzalez'),
('Jane', 'Doe'),
('Emily', 'Johnson'),
('Mark', 'Brown'),
('Sophia', 'Davis'),
('David', 'Miller'),
('Sarah', 'Wilson'),
('Michael', 'Taylor'),
('Laura', 'Garcia'),
('James', 'Martinez');
SELECT * FROM Author


Running query in 'sqlite:///library2.db'

19 rows affected.

firstName,lastName
John,Smith
Mary,Johnson
James,Williams
Patricia,Brown
Michael,Jones
Linda,Garcia
David,Martinez
Susan,Hernandez
Robert,Lopez
Elizabeth,Gonzalez


In [4]:
%%sql
INSERT INTO LibraryItem (itemID, title, type, publicationDate, authorFirstName, authorLastName, isFutureAcq, expectedAcquisitionDate) VALUES
(1, 'Introduction to Programming', 'Book', '2023-02-15', 'John', 'Smith', 0, NULL),
(2, 'Advanced Java Concepts', 'Book', '2022-08-01', 'Mary', 'Johnson', 0, NULL),
(3, 'Database Design', 'Book', '2020-12-05', 'James', 'Williams', 0, NULL),
(4, 'Machine Learning 101', 'Book', '2021-06-20', 'Patricia', 'Brown', 0, NULL),
(5, 'React for Beginners', 'Book', '2023-01-01', 'Michael', 'Jones', 0, NULL),
(6, 'Web Development Best Practices', 'Book', '2022-04-12', 'Linda', 'Garcia', 0, NULL),
(7, 'Data Science Handbook', 'Book', '2021-11-30', 'David', 'Martinez', 0, NULL),
(8, 'Python Programming', 'Book', '2023-03-10', 'Susan', 'Hernandez', 0, NULL),
(9, 'Digital Transformation', 'Book', '2022-07-17', 'Robert', 'Lopez', 1, '2024-01-01'),
(10, 'Cloud Computing', 'Book', '2022-10-10', 'Elizabeth', 'Gonzalez', 1, '2024-05-01'),
(11, 'Advanced Java Concepts', 'Book', '2023-03-01', 'John', 'Smith', 0, NULL),
(12, 'Introduction to Machine Learning', 'Book', '2022-06-15', 'Jane', 'Doe', 0, NULL),
(13, 'React for Beginners', 'Book', '2023-01-20', 'Emily', 'Johnson', 1, '2023-08-01'),
(14, 'Cloud Computing Essentials', 'Book', '2022-09-10', 'Mark', 'Brown', 0, NULL),
(15, 'Web Development in 2023', 'Magazine', '2023-03-05', 'Sophia', 'Davis', 0, NULL),
(16, 'Python Programming Guide', 'Book', '2023-04-12', 'David', 'Miller', 1, '2023-10-01'),
(17, 'Data Science with R', 'Book', '2022-11-25', 'Sarah', 'Wilson', 0, NULL),
(18, 'AI and Deep Learning', 'Journal', '2023-02-14', 'Michael', 'Taylor', 0, NULL),
(19, 'Digital Transformation for Businesses', 'Book', '2022-12-22', 'James', 'Martinez', 1, '2023-09-01'),
(20, 'Innovations in Cloud Security', 'Book', '2023-03-25', 'Laura', 'Garcia', 0, NULL);
SELECT * FROM LibraryItem

Running query in 'sqlite:///library2.db'

20 rows affected.

itemID,title,type,publicationDate,authorFirstName,authorLastName,isFutureAcq,expectedAcquisitionDate
1,Introduction to Programming,Book,2023-02-15,John,Smith,0,None
2,Advanced Java Concepts,Book,2022-08-01,Mary,Johnson,0,None
3,Database Design,Book,2020-12-05,James,Williams,0,None
4,Machine Learning 101,Book,2021-06-20,Patricia,Brown,0,None
5,React for Beginners,Book,2023-01-01,Michael,Jones,0,None
6,Web Development Best Practices,Book,2022-04-12,Linda,Garcia,0,None
7,Data Science Handbook,Book,2021-11-30,David,Martinez,0,None
8,Python Programming,Book,2023-03-10,Susan,Hernandez,0,None
9,Digital Transformation,Book,2022-07-17,Robert,Lopez,1,2024-01-01
10,Cloud Computing,Book,2022-10-10,Elizabeth,Gonzalez,1,2024-05-01


In [5]:
%%sql
INSERT INTO User (libraryCardNumber, address, firstName, lastName, email, phoneNumber) VALUES
(101, '123 Maple St, Vancouver, BC', 'Alice', 'Green', 'alice.green@example.com', '123-456-7890'),
(102, '456 Oak Rd, Toronto, ON', 'Bob', 'White', 'bob.white@example.com', '234-567-8901'),
(103, '789 Pine Ave, Calgary, AB', 'Charlie', 'Black', 'charlie.black@example.com', '345-678-9012'),
(104, '321 Birch Ln, Montreal, QC', 'David', 'Blue', 'david.blue@example.com', '456-789-0123'),
(105, '654 Cedar Blvd, Ottawa, ON', 'Eva', 'Brown', 'eva.brown@example.com', '567-890-1234'),
(106, '987 Elm St, Edmonton, AB', 'Frank', 'Yellow', 'frank.yellow@example.com', '678-901-2345'),
(107, '111 Fir Dr, Calgary, AB', 'Grace', 'Red', 'grace.red@example.com', '789-012-3456'),
(108, '222 Maple Ave, Vancouver, BC', 'Helen', 'Gray', 'helen.gray@example.com', '890-123-4567'),
(109, '333 Oak Rd, Toronto, ON', 'Ivan', 'Blue', 'ivan.blue@example.com', '901-234-5678'),
(110, '444 Pine Ln, Montreal, QC', 'Jack', 'Pink', 'jack.pink@example.com', '012-345-6789'),
(201, '123 Maple Street, Toronto, ON', 'Sarah', 'Johnson', 'sarah.johnson@email.com', '416-555-1234'),
(202, '456 Oak Avenue, Vancouver, BC', 'James', 'Brown', 'james.brown@email.com', '604-555-5678'),
(203, '789 Pine Road, Calgary, AB', 'Emily', 'Davis', 'emily.davis@email.com', '403-555-8765'),
(204, '101 Birch Lane, Ottawa, ON', 'Michael', 'Miller', 'michael.miller@email.com', '613-555-4321'),
(205, '202 Cedar Drive, Edmonton, AB', 'Jessica', 'Wilson', 'jessica.wilson@email.com', '780-555-9876'),
(206, '303 Elm Street, Montreal, QC', 'David', 'Moore', 'david.moore@email.com', '514-555-6543'),
(207, '404 Maple Avenue, Winnipeg, MB', 'Laura', 'Taylor', 'laura.taylor@email.com', '204-555-3456'),
(208, '505 Birch Road, Calgary, AB', 'Daniel', 'Anderson', 'daniel.anderson@email.com', '403-555-2345'),
(209, '606 Oak Lane, Toronto, ON', 'Sophia', 'Thomas', 'sophia.thomas@email.com', '416-555-6789'),
(210, '707 Pine Street, Vancouver, BC', 'John', 'Jackson', 'john.jackson@email.com', '604-555-3456');
SELECT * FROM User

Running query in 'sqlite:///library2.db'

20 rows affected.

libraryCardNumber,address,firstName,lastName,email,phoneNumber
101,"123 Maple St, Vancouver, BC",Alice,Green,alice.green@example.com,123-456-7890
102,"456 Oak Rd, Toronto, ON",Bob,White,bob.white@example.com,234-567-8901
103,"789 Pine Ave, Calgary, AB",Charlie,Black,charlie.black@example.com,345-678-9012
104,"321 Birch Ln, Montreal, QC",David,Blue,david.blue@example.com,456-789-0123
105,"654 Cedar Blvd, Ottawa, ON",Eva,Brown,eva.brown@example.com,567-890-1234
106,"987 Elm St, Edmonton, AB",Frank,Yellow,frank.yellow@example.com,678-901-2345
107,"111 Fir Dr, Calgary, AB",Grace,Red,grace.red@example.com,789-012-3456
108,"222 Maple Ave, Vancouver, BC",Helen,Gray,helen.gray@example.com,890-123-4567
109,"333 Oak Rd, Toronto, ON",Ivan,Blue,ivan.blue@example.com,901-234-5678
110,"444 Pine Ln, Montreal, QC",Jack,Pink,jack.pink@example.com,012-345-6789


In [6]:
%%sql
INSERT INTO LibraryStaff (staffID, staffCardNumber, hireDate, salary) VALUES
(1, 101, '2021-01-15', 55000.00),
(2, 102, '2020-06-30', 60000.00),
(3, 103, '2022-03-05', 52000.00),
(4, 104, '2021-10-01', 58000.00),
(5, 105, '2023-02-20', 63000.00),
(6, 106, '2022-07-18', 54000.00),
(7, 107, '2020-12-10', 59000.00),
(8, 108, '2023-05-01', 62000.00),
(9, 109, '2021-11-11', 57000.00),
(10, 110, '2022-09-25', 55000.00);
SELECT * FROM LibraryStaff

Running query in 'sqlite:///library2.db'

10 rows affected.

staffID,staffCardNumber,hireDate,salary
1,101,2021-01-15,55000
2,102,2020-06-30,60000
3,103,2022-03-05,52000
4,104,2021-10-01,58000
5,105,2023-02-20,63000
6,106,2022-07-18,54000
7,107,2020-12-10,59000
8,108,2023-05-01,62000
9,109,2021-11-11,57000
10,110,2022-09-25,55000


In [7]:
%%sql
INSERT INTO Loan (loanCard, itemID, loanDateTime, dueDate, returnDate, renewalCount) VALUES
(201, 1, '2023-03-10 10:00:00', '2023-04-10', '2023-04-08', 1),
(202, 2, '2023-02-15 14:00:00', '2023-03-15', '2023-03-14', 0),
(203, 3, '2023-03-01 09:30:00', '2023-04-01', '2023-04-05', 0),
(204, 4, '2023-01-20 11:00:00', '2023-02-20', '2023-02-18', 1),
(105, 5, '2023-03-12 16:00:00', '2023-04-12', NULL, 0),
(206, 6, '2023-03-05 12:30:00', '2023-04-05', '2023-04-03', 1),
(107, 7, '2023-02-28 08:00:00', '2023-03-28', '2023-03-25', 0),
(108, 8, '2023-03-08 15:45:00', '2023-04-08', NULL, 0),
(209, 9, '2023-03-20 13:00:00', '2023-04-20', NULL, 0),
(110, 10, '2023-02-22 17:00:00', '2023-03-22', '2023-03-19', 2);
SELECT * FROM Loan

Running query in 'sqlite:///library2.db'

10 rows affected.

loanCard,itemID,loanDateTime,dueDate,returnDate,renewalCount
201,1,2023-03-10 10:00:00,2023-04-10,2023-04-08,1
202,2,2023-02-15 14:00:00,2023-03-15,2023-03-14,0
203,3,2023-03-01 09:30:00,2023-04-01,2023-04-05,0
204,4,2023-01-20 11:00:00,2023-02-20,2023-02-18,1
105,5,2023-03-12 16:00:00,2023-04-12,None,0
206,6,2023-03-05 12:30:00,2023-04-05,2023-04-03,1
107,7,2023-02-28 08:00:00,2023-03-28,2023-03-25,0
108,8,2023-03-08 15:45:00,2023-04-08,None,0
209,9,2023-03-20 13:00:00,2023-04-20,None,0
110,10,2023-02-22 17:00:00,2023-03-22,2023-03-19,2


In [8]:
%%sql
INSERT INTO Fine (fineLibraryCard, dateTimeIssued, amount, paid) VALUES
(201, '2023-03-12 14:30:00', 5.00, 0),
(202, '2023-03-01 10:00:00', 2.50, 1),
(203, '2023-02-28 16:00:00', 10.00, 0),
(204, '2023-03-03 09:15:00', 7.00, 1),
(105, '2023-03-04 18:30:00', 15.00, 0),
(206, '2023-03-06 12:00:00', 8.00, 1),
(107, '2023-03-07 11:30:00', 3.00, 0),
(108, '2023-03-02 10:15:00', 6.50, 0),
(209, '2023-03-08 14:45:00', 4.00, 0),
(110, '2023-03-09 13:00:00', 12.00, 1);
SELECT * FROM Fine

Running query in 'sqlite:///library2.db'

10 rows affected.

fineLibraryCard,dateTimeIssued,amount,paid
201,2023-03-12 14:30:00,5,0
202,2023-03-01 10:00:00,2.5,1
203,2023-02-28 16:00:00,10,0
204,2023-03-03 09:15:00,7,1
105,2023-03-04 18:30:00,15,0
206,2023-03-06 12:00:00,8,1
107,2023-03-07 11:30:00,3,0
108,2023-03-02 10:15:00,6.5,0
209,2023-03-08 14:45:00,4,0
110,2023-03-09 13:00:00,12,1


In [9]:
%%sql
INSERT INTO LibraryRoom (RoomNumber, capacity) VALUES
(101, 50),
(102, 100),
(103, 25),
(104, 75),
(105, 150),
(106, 200),
(107, 20),
(108, 30),
(109, 120),
(110, 60);
SELECT * FROM LibraryRoom

Running query in 'sqlite:///library2.db'

10 rows affected.

RoomNumber,capacity
101,50
102,100
103,25
104,75
105,150
106,200
107,20
108,30
109,120
110,60


In [10]:
%%sql
INSERT INTO Event (title, dateTime, roomNum, numAttendees, maxAttendees, duration, registrationEnd, description) VALUES
('Book Launch: Java Programming', '2023-04-01 10:00:00', 101, 1, 50, 120, '2023-03-31 23:59:59', 'A book launch event for "Advanced Java Concepts"'),
('Machine Learning Workshop', '2023-04-05 13:00:00', 102, 1, 100, 180, '2023-04-04 23:59:59', 'A workshop on machine learning algorithms'),
('React for Beginners', '2023-04-10 09:00:00', 103, 1, 5, 90, '2023-04-09 23:59:59', 'Introduction to React for new developers'),
('Web Development Conference', '2023-04-15 11:00:00', 104, 1, 75, 240, '2023-04-14 23:59:59', 'A conference discussing modern web development'),
('Python Programming Bootcamp', '2023-04-20 14:00:00', 105, 1, 50, 120, '2023-04-19 23:59:59', 'Bootcamp for Python programming enthusiasts'),
('Data Science Meetup', '2023-04-25 16:00:00', 106, 1, 30, 180, '2023-04-24 23:59:59', 'A meetup for data science professionals'),
('Digital Transformation Seminar', '2023-05-01 10:00:00', 107, 1, 60, 150, '2023-04-30 23:59:59', 'A seminar on digital transformation strategies'),
('Library Tech Talk', '2023-05-10 09:30:00', 108, 1, 30, 60, '2023-05-09 23:59:59', 'Tech talk on innovations in library technology'),
('Cloud Computing Workshop', '2023-05-15 13:00:00', 109, 1, 120, 180, '2023-05-14 23:59:59', 'A workshop on cloud computing infrastructure'),
('AI for Beginners', '2023-05-20 14:00:00', 110, 1, 50, 120, '2023-05-19 23:59:59', 'Introduction to Artificial Intelligence concepts');
SELECT * FROM Event

Running query in 'sqlite:///library2.db'

10 rows affected.

title,dateTime,roomNum,numAttendees,maxAttendees,duration,registrationEnd,description
Book Launch: Java Programming,2023-04-01 10:00:00,101,1,50,120,2023-03-31 23:59:59,"A book launch event for ""Advanced Java Concepts"""
Machine Learning Workshop,2023-04-05 13:00:00,102,1,100,180,2023-04-04 23:59:59,A workshop on machine learning algorithms
React for Beginners,2023-04-10 09:00:00,103,1,5,90,2023-04-09 23:59:59,Introduction to React for new developers
Web Development Conference,2023-04-15 11:00:00,104,1,75,240,2023-04-14 23:59:59,A conference discussing modern web development
Python Programming Bootcamp,2023-04-20 14:00:00,105,1,50,120,2023-04-19 23:59:59,Bootcamp for Python programming enthusiasts
Data Science Meetup,2023-04-25 16:00:00,106,1,30,180,2023-04-24 23:59:59,A meetup for data science professionals
Digital Transformation Seminar,2023-05-01 10:00:00,107,1,60,150,2023-04-30 23:59:59,A seminar on digital transformation strategies
Library Tech Talk,2023-05-10 09:30:00,108,1,30,60,2023-05-09 23:59:59,Tech talk on innovations in library technology
Cloud Computing Workshop,2023-05-15 13:00:00,109,1,120,180,2023-05-14 23:59:59,A workshop on cloud computing infrastructure
AI for Beginners,2023-05-20 14:00:00,110,1,50,120,2023-05-19 23:59:59,Introduction to Artificial Intelligence concepts


In [11]:
%%sql
INSERT INTO EventAttendees (libraryCardNumber, eventTitle, eventDateTime) VALUES
(201, 'Book Launch: Java Programming', '2023-04-01 10:00:00'),
(202, 'Machine Learning Workshop', '2023-04-05 13:00:00'),
(203, 'React for Beginners', '2023-04-10 09:00:00'),
(204, 'Web Development Conference', '2023-04-15 11:00:00'),
(205, 'Python Programming Bootcamp', '2023-04-20 14:00:00'),
(206, 'Data Science Meetup', '2023-04-25 16:00:00'),
(207, 'Digital Transformation Seminar', '2023-05-01 10:00:00'),
(208, 'Library Tech Talk', '2023-05-10 09:30:00'),
(209, 'Cloud Computing Workshop', '2023-05-15 13:00:00'),
(210, 'AI for Beginners', '2023-05-20 14:00:00');
SELECT * FROM EventAttendees

Running query in 'sqlite:///library2.db'

10 rows affected.

libraryCardNumber,eventTitle,eventDateTime
201,Book Launch: Java Programming,2023-04-01 10:00:00
202,Machine Learning Workshop,2023-04-05 13:00:00
203,React for Beginners,2023-04-10 09:00:00
204,Web Development Conference,2023-04-15 11:00:00
205,Python Programming Bootcamp,2023-04-20 14:00:00
206,Data Science Meetup,2023-04-25 16:00:00
207,Digital Transformation Seminar,2023-05-01 10:00:00
208,Library Tech Talk,2023-05-10 09:30:00
209,Cloud Computing Workshop,2023-05-15 13:00:00
210,AI for Beginners,2023-05-20 14:00:00


In [12]:
%%sql

DROP TABLE IF EXISTS LibraryItem;
DROP TABLE IF EXISTS LibraryRoom;
DROP TABLE IF EXISTS LibraryStaff;
DROP TABLE IF EXISTS Fine;
DROP TABLE IF EXISTS Loan;
DROP TABLE IF EXISTS Author;
DROP TABLE IF EXISTS User;
DROP TABLE Event;
DROP TABLE IF EXISTS EventAttendees;



Running query in 'sqlite:///library2.db'

RuntimeError: (sqlite3.OperationalError) database table is locked
[SQL: DROP TABLE IF EXISTS LibraryItem;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community
